<a href="https://colab.research.google.com/github/JuanFelipePosada/SenalesSistemas/blob/master/parcial_2_2024_DASHBOARD_corregido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install streamlit -q
!pip install pyngrok
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile
!pip install -U yt-dlp

  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz (2.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached websockets-13.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Usi

In [63]:
%%writefile app.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import tempfile
from scipy.fftpack import fft
from scipy.signal import square
from scipy.interpolate import interp1d

# Configuración inicial de la página
st.set_page_config(page_title="Parcial 2 - Fourier SyS", page_icon="📊", layout="wide")

# Barra lateral para seleccionar el punto del parcial y mostrar información general
with st.sidebar:
    st.image("https://lh4.googleusercontent.com/proxy/WNtyuTbDjnnITJFxg1dlI63L0jfIMRf0CIKg75VavFd3ameUuokpEiXIZvafO0UbA3rGKkhjDZ2HFtRWcGiPIn7Syd37PqnCrQuXFNHguRRPYm__safRJi9Q", width=150)
    st.title("Parcial 2 - Fourier SyS")
    st.subheader("Universidad Nacional de Colombia - Sede Manizales")
    st.write("**Señales y Sistemas - 2024-1**")
    st.write("**Integrantes:**")
    st.write("- Juan Felipe Posada Niasa")
    st.write("- Juan Jose Montilla Bastidas")

    # Menú desplegable para seleccionar el punto del parcial
    punto = st.selectbox("Selecciona el punto del parcial",
                         ("Punto 1: Expresión del Espectro de Fourier",
                          "Punto 2: Simulación y Espectro de Fourier",
                          "Punto 3: Modulación en Amplitud (AM) con Audio",
                          "Punto 4: THD y Factor de Potencia"))



# Mostrar información general del punto seleccionado
if punto == "Punto 1: Expresión del Espectro de Fourier":

        st.write("### Expresión del Espectro de Fourier (Forma Exponencial y Trigonométrica) ")

        # Desarrollo de la señal
        st.write("#### Desarrollo:")
        st.write("Dada la señal original:")
        st.latex(r"x(t) = |A \sin(2\pi F_o t)|^2 = A^2 \sin^2(2\pi F_o t) = A^2 \left( \frac{1}{2} - \frac{\cos(2\pi 2 F_o t)}{2} \right)")

        st.write("Aplicando la serie trigonométrica:")
        st.latex(r"x(t) = a_0 + \sum_{n=-N}^N a_n \cos(n\omega_0 t) + b_n \sin(n\omega_0 t)")
        st.write("Como la señal es par, \(b_n = 0\) para todos los \(n\). Los coeficientes son:")
        st.latex(r"a_0 = \frac{A^2}{2}")
        st.latex(r"a_2 = -\frac{A^2}{2}")
        st.latex(r"a_n = 0 \quad \text{para} \quad n \neq 2")

        st.write("Para la serie exponencial compleja:")
        st.latex(r"x(t) = \sum_{n=-N}^{N} c_n e^{j n \omega_0 t}")
        st.write("Donde:")
        st.latex(r"c_0 = \frac{A^2}{2}")
        st.latex(r"c_2 = -\frac{A^2}{4}")
        st.latex(r"c_{-2} = \frac{A^2}{4}")

        st.write("## Parametros para de la señal")

        A = st.slider("Amplitud de señal(A)", 1, 11, 0)
        F_o= st.slider("Frecuencia de señal original(F)", 0, 120  , step=10)
        ti= -1/ (2 * F_o)
        tf= 1/ (2 * F_o)
        Nm= 1000
        tv = np.linspace(ti, tf, Nm)
        x = np.abs(A * np.sin(2 * np.pi * F_o * tv))**2

        # Graficar la señal
        st.write("### Gráfico de la señal $|A\\sin(2\\pi F_o t)|^2$")
        fig, ax = plt.subplots()
        ax.plot(tv, x)
        ax.set_xlabel('Tiempo [s]')
        ax.set_ylabel('$x(t)$')
        ax.grid(True)
        st.pyplot(fig)

#######################################################################################

elif punto == "Punto 2: Simulación y Espectro de Fourier":
        st.write("### Simulacion y Espectros de Fourier")
        st.write("## ahora graficamos los espectros, fase, magnitud de la serie de fourier que hallamos en el punto 1")


Overwriting app.py


In [64]:
from pyngrok import ngrok

# Token personal de ngrok
token = "2lLFiUsTGsQClQyxWz9mY9rLIup_7vUusFxPVHjfdMEa1CryT"
ngrok.set_auth_token(token)

# Ejecutar Streamlit en el puerto 5011
!nohup streamlit run app.py --server.port 5011 &

# Iniciar el túnel Ngrok y exponer Streamlit en una URL.
public_url = ngrok.connect(5011)
print(f"Aplicación aquí: {public_url}")

nohup: appending output to 'nohup.out'
Aplicación aquí: NgrokTunnel: "https://49da-34-106-57-91.ngrok-free.app" -> "http://localhost:5011"
